In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import scipy
from scipy.stats import norm
import tensorflow_probability as tfp
import seaborn as sns
import pandas as pd
import corner 
tfd=tfp.distributions

In [2]:
depth = -20
thickness = 20

corners=np.array([[-200,depth-thickness],
                  [-70,depth-thickness],
                  [70,depth-thickness],# bottom left coner
              [200,depth-thickness], 
              [200,depth], # bottom right coner
              [70, depth], # top right coner
              [-70,depth],# top left coner
              [-200,depth]])
N = 2
points = np.empty((2*N+8,2),dtype = np.float32)

points[0] = corners[0]
points[1] = corners[1]

for i in range(6):
    points[N+i] = corners[i]
    
points[-2]=corners[-2]
points[-1]=corners[-1]

np.random.seed(11)

x2 = np.linspace(-70,70,N+2)
x1 = np.linspace(70,-70,N+2)

thickness = 20
y1 = np.random.uniform(-2,-50,N)
y2 = y1-thickness

y1 = list(reversed(y1))

for i in range(2,N+2): ## lower layer
    points[i][1] = y2[i-2]
    points[i][0] = x2[i-1]
    
for i in range(N+6,2*N+6): ## upper layer
    points[i][1] = y1[i-N-6]
    points[i][0] = x1[i-N-5]


In [193]:
@tf.function
def A(x,z,p1,p2):
    numerator = (x[p2]-x[p1])*(x[p1]*z[p2]-x[p2]*z[p1])
    denominator = (x[p2]-x[p1])**2 + (z[p2]-z[p1])**2
    return (numerator/denominator)

@tf.function
def B(x,z,p1,p2):
    return ((z[p1]-z[p2])/(x[p2]-x[p1]))

@tf.function
def theta(x,z, p):
    if tf.math.not_equal(x[p], 0) :
        if tf.less(tf.atan(z[p]/x[p]),0):
            return(tf.atan(z[p]/x[p])+scipy.pi)
        else:
            return(tf.atan(z[p]/x[p]))
    elif tf.math.logical_and(tf.math.equal(x[p], 0), tf.math.not_equal(z[p], 0)):
        return(scipy.pi/2)
    else: return(0.)

@tf.function
def r(x,z,p):
    return(tf.sqrt(x[p]**2+z[p]**2))

@tf.function
def Z(x,z,p1,p2):
    
    if tf.logical_or(tf.logical_and(tf.equal(x[p1],z[p1]),tf.equal(x[p1],0.)), tf.logical_and(tf.equal(x[p2],z[p2]),tf.equal(x[p2],0.))):
        return(0.)

    elif tf.equal(x[p1], x[p2]):
        return((x[p1]*tf.math.log(r(x,z,p2)/r(x,z,p1))))
    
    else:
    
        theta1 = theta(x,z, p1)
        theta2 = theta(x,z, p2)

        r1 = r(x,z,p1)
        r2 = r(x,z,p2)

        _A = A(x,z,p1,p2)
        _B = B(x,z,p1,p2)

        Z_result = _A*((theta1-theta2)+_B*tf.math.log(r1/r2))
        return(Z_result)

@tf.function
def g(data,loc):
    
    G = tf.constant(6.67 * 10**(-11)) # gravitational constant  m^3 kg ^-1 s^-2
    rho = tf.constant(1000.)        # density difference   kg/m^3

    _data = points - loc #Calculate any point refer to the origin

    _x = _data[:,0]
    _z = _data[:,1]

    Z_sum = tf.constant(0.)

    for i in range(_data.shape[0]-1):
        Z_sum = tf.add(Z_sum, Z(_x,_z,i,i+1))

    Z_sum = tf.add(Z_sum, Z(_x,_z,-1,0))

    g = 2*G*rho * Z_sum

    return(g)

@tf.function
def grav(base,ps):
    ps = tf.constant(ps)

    ps1 = ps
    ps2 = ps1-thickness

    ps2 = tf.reverse(ps2,[-1])

    for i in range(2,N+2): ## lower layer
        points[i].assign([points[i][0],ps2[i-2]])

    for i in range(N+6,2*N+6): ## upper layer
        points[i].assign([points[i][0],ps1[i-N-6]])

    x_obv = tf.linspace(-70., 70., 11)
    y_obv = tf.zeros(tf.shape(x_obv))
    obv = tf.stack((x_obv,y_obv),axis = 1)


    gravity = tf.TensorArray(tf.float32, size=obv.shape[0]-2)
    
    j = tf.constant(0)
    for i in obv[1:-1]:
        gravity=gravity.write(j,-g(points,i))
        j = tf.add(j,1)
    return gravity.stack()


@tf.function
def joint_log_prob(D,points_copy,ps):
    """
    D: is the observation data
    ps: is the variable point positions (N elements vector)
    """
    # define random variables prior
    
    low_ = tf.constant(-40.)
    high_ = tf.constant(-1.)
    mvn_prior = tfd.Uniform(
            low = low_,
            high = high_)
    # define likelihood
    
    Gm_ = grav(points_copy,ps)
    
    mvn_likelihood = tfd.MultivariateNormalFullCovariance(
            loc = Gm_,
            covariance_matrix= cov)
    
    # return the posterior probability
    return(tf.reduce_sum(mvn_prior.log_prob(ps))
          +mvn_likelihood.log_prob(D))

In [ ]:
mu_prior = -20.*tf.ones([50],dtype = tf.float32)
cov_prior = 20.*tf.eye(N)

sig_e = 0.0000003
cov = sig_e**2*tf.eye(np.shape(obs_data)[0])

In [4]:
N = 2

In [5]:
x = [2.,2.,3.,4.]
z = [0.,1.,-3.,-4.]
x = tf.convert_to_tensor(x,dtype = tf.float32)
z = tf.convert_to_tensor(z,dtype = tf.float32)

In [206]:
@tf.function
def grav(base,ps):
    ps = tf.constant(ps)

    ps1 = ps
    ps2 = ps1-thickness

    ps2 = tf.reverse(ps2,[-1])

    for i in range(2,N+2): ## lower layer
        points[i].assign([points[i][0],ps2[i-2]])

    for i in range(N+6,2*N+6): ## upper layer
        points[i].assign([points[i][0],ps1[i-N-6]])

    x_obv = tf.linspace(-70., 70., 11)
    y_obv = tf.zeros(tf.shape(x_obv))
    obv = tf.stack((x_obv,y_obv),axis = 1)


    gravity = tf.TensorArray(tf.float32, size=obv.shape[0]-2)
    
    j = tf.constant(0)
    for i in obv[1:-1]:
        gravity=gravity.write(j,-g(points,i))
        j = tf.add(j,1)
    return gravity.stack()


In [207]:
obs_data = grav(points,[-1.,-2,])

In [191]:
points = tf.Variable(points,dtype = tf.float32)

In [228]:
D = tf.constant([7.3171241e-06, 7.9643241e-06, 8.9585574e-06, 9.4861543e-06,9.4992556e-06, 9.6535805e-06, 9.1502252e-06, 8.0245300e-06,7.3247274e-06])

In [229]:
joint_log_prob(D,points,[-10.,-2.])

<tf.Tensor: id=34291, shape=(), dtype=float32, numpy=100.59033>